# LAB 1.5: CROSS-LLM PROMPT PORTABILITY

**Course:** Advanced Prompt Engineering Training  
**Session:** Session 1 - Prompt Engineering Fundamentals Review  
**Duration:** 50 minutes  
**Type:** Hands-on Multi-Provider Integration

## LAB OVERVIEW

This lab explores **cross-LLM prompt portability** - designing prompts that work reliably across different model providers (OpenAI, Anthropic, Azure, etc.). You'll learn to:

- Understand model-specific behaviors and quirks
- Adapt prompts for different providers
- Build provider-agnostic abstraction layers
- Test prompts across multiple models
- Design fallback and failover strategies

**Scenario:** Your bank is evaluating multiple LLM providers for different use cases. Some departments use OpenAI (GPT-4), others use Anthropic (Claude), and compliance requires Azure OpenAI for data residency. You need to ensure prompts work consistently across all providers while minimizing vendor lock-in.

## LEARNING OBJECTIVES

By the end of this lab, you will be able to:

✓ Identify model-specific behaviors and limitations  
✓ Adapt prompts for different LLM providers  
✓ Build provider abstraction layers  
✓ Create unified testing frameworks  
✓ Design multi-provider fallback strategies  
✓ Maintain prompt quality across models

### Step 1: Import Libraries

In [ ]:
# Lab 1.5: Cross-LLM Prompt Portability
# Advanced Prompt Engineering Training - Session 1

import os
import json
from openai import OpenAI
from typing import Dict, List, Any, Optional, Callable
import pandas as pd
from abc import ABC, abstractmethod
import re
from dotenv import load_dotenv

load_dotenv(override=True)  # Load environment variables from .env file

# Try to import Anthropic (optional)
try:
    from anthropic import Anthropic
    ANTHROPIC_AVAILABLE = True
except ImportError:
    ANTHROPIC_AVAILABLE = False
    print("⚠ Anthropic SDK not available - some examples will be skipped")

print("✓ Libraries imported")

### Step 2: Configure API Clients

In [ ]:
# Initialize OpenAI client
api_key=os.environ.get("OPENAI_API_KEY")

# Configuration
MODEL = os.getenv("MODEL_NAME")
TEMPERATURE = 0  # Deterministic for consistent debugging

if not api_key:
    raise ValueError("OPENAI_API_KEY not found. Please set it in .env file")

if not MODEL:
    raise ValueError("MODEL_NAME not found. Please set it in .env file")

openai_client = OpenAI(api_key=api_key)

print(f"✓ Model: {MODEL}")
print(f"✓ Temperature: {TEMPERATURE}")

# Initialize Anthropic client (optional)
anthropic_client = None
if ANTHROPIC_AVAILABLE and os.environ.get("ANTHROPIC_API_KEY"):
    anthropic_client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))
    print("✓ Anthropic client configured")
else:
    print("ℹ Anthropic client not available (optional)")

print("✓ OpenAI client configured")

### Step 3: Create Helper Functions

In [ ]:
def call_openai(
    prompt: str, 
    system_prompt: str = "You are a helpful AI assistant.",
    model: str = "gpt-4",
    temperature: float = 0
) -> Dict:
    """
    Call OpenAI API
    
    Args:
        prompt (str): User prompt
        system_prompt (str): System prompt
        model (str): Model name
        temperature (float): Sampling temperature
    
    Returns:
        Dict: Response with metadata
    """
    try:
        response = openai_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature
        )
        
        return {
            "provider": "openai",
            "model": model,
            "content": response.choices[0].message.content,
            "total_tokens": response.usage.total_tokens,
            "finish_reason": response.choices[0].finish_reason,
            "success": True
        }
    except Exception as e:
        return {
            "provider": "openai",
            "model": model,
            "content": "",
            "error": str(e),
            "success": False
        }

def call_anthropic(
    prompt: str,
    system_prompt: str = "You are a helpful AI assistant.",
    model: str = "claude-3-5-sonnet-20241022",
    temperature: float = 0
) -> Dict:
    """
    Call Anthropic API
    
    Args:
        prompt (str): User prompt
        system_prompt (str): System prompt
        model (str): Model name
        temperature (float): Sampling temperature
    
    Returns:
        Dict: Response with metadata
    """
    if not anthropic_client:
        return {
            "provider": "anthropic",
            "model": model,
            "content": "",
            "error": "Anthropic client not available",
            "success": False
        }
    
    try:
        response = anthropic_client.messages.create(
            model=model,
            max_tokens=4096,
            system=system_prompt,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=temperature
        )
        
        return {
            "provider": "anthropic",
            "model": model,
            "content": response.content[0].text,
            "total_tokens": response.usage.input_tokens + response.usage.output_tokens,
            "finish_reason": response.stop_reason,
            "success": True
        }
    except Exception as e:
        return {
            "provider": "anthropic",
            "model": model,
            "content": "",
            "error": str(e),
            "success": False
        }

print("✓ Helper functions created")

### Step 4: Test Connections

In [ ]:
# Test OpenAI
print("Testing OpenAI connection...")
test_openai = call_openai("Say 'OpenAI ready' if you receive this.")
if test_openai['success']:
    print(f"✓ OpenAI: {test_openai['content']}")
else:
    print(f"✗ OpenAI error: {test_openai.get('error', 'Unknown')}")

# Test Anthropic (if available)
if anthropic_client:
    print("\nTesting Anthropic connection...")
    test_anthropic = call_anthropic("Say 'Anthropic ready' if you receive this.")
    if test_anthropic['success']:
        print(f"✓ Anthropic: {test_anthropic['content']}")
    else:
        print(f"✗ Anthropic error: {test_anthropic.get('error', 'Unknown')}")

print("\n✓ Connection tests complete")

## CROSS-LLM CHALLENGES

### Why Multi-Provider Matters

**Business Reasons:**
- **Vendor diversification** - Avoid single-provider lock-in
- **Cost optimization** - Use cheaper models for simpler tasks
- **Compliance** - Data residency requirements (Azure OpenAI in EU)
- **Capabilities** - Different models excel at different tasks
- **Redundancy** - Failover if one provider has downtime

**Technical Challenges:**
- Different API formats
- Different prompt behaviors
- Different token limits
- Different pricing structures
- Different rate limits
- Different output formats

## CHALLENGE 1: MODEL BEHAVIOR DIFFERENCES

**Time:** 10 minutes  
**Objective:** Identify and document model-specific behaviors

### Background

The same prompt can produce different outputs across models. Some differences are subtle, others are significant.

### Test Case: JSON Output

In [ ]:
# Test JSON output consistency across models

json_prompt = """
Extract the following information from this transaction and return as JSON:

Transaction: Card 4523, $1,250 purchase at Electronics Store, Lagos Nigeria, 3:47 AM

Required fields:
- card_last_4
- amount
- merchant
- location
- time
- risk_level (HIGH, MEDIUM, or LOW)

Return ONLY valid JSON, no additional text.
"""

system_prompt = "You are a data extraction assistant."

print("JSON OUTPUT TEST:")
print("=" * 80)

# Test with OpenAI
openai_result = call_openai(json_prompt, system_prompt)
print(f"\nOpenAI GPT-4o:")
print(f"Output:\n{openai_result['content']}")
print(f"Success: {openai_result['success']}")

# Test with Anthropic (if available)
if anthropic_client:
    anthropic_result = call_anthropic(json_prompt, system_prompt)
    print(f"\nAnthropic Claude:")
    print(f"Output:\n{anthropic_result['content']}")
    print(f"Success: {anthropic_result['success']}")

print("\n" + "=" * 80)

### Test Case: Instruction Following

In [ ]:
# Test strict instruction following

strict_prompt = """
You must answer with EXACTLY one word: yes or no.

Question: Is 2+2 equal to 4?

Remember: EXACTLY one word only.
"""

print("\nSTRICT INSTRUCTION TEST:")
print("=" * 80)

# Test with OpenAI
openai_strict = call_openai(strict_prompt, "Follow instructions precisely.")
print(f"\nOpenAI Response: '{openai_strict['content']}'")
print(f"Word count: {len(openai_strict['content'].split())}")

# Test with Anthropic
if anthropic_client:
    anthropic_strict = call_anthropic(strict_prompt, "Follow instructions precisely.")
    print(f"\nAnthropic Response: '{anthropic_strict['content']}'")
    print(f"Word count: {len(anthropic_strict['content'].split())}")

print("=" * 80)

### Documented Differences

In [ ]:
# Comprehensive behavior matrix

behavior_differences = pd.DataFrame([
    {
        "Aspect": "JSON Formatting",
        "OpenAI": "May add ```json markers",
        "Anthropic": "May add preambles",
        "Solution": "Explicit 'no markdown' instruction"
    },
    {
        "Aspect": "Output Length",
        "OpenAI": "More concise by default",
        "Anthropic": "More verbose by default",
        "Solution": "Specify exact word/character limits"
    },
    {
        "Aspect": "Instruction Adherence",
        "OpenAI": "Good with clear instructions",
        "Anthropic": "Excellent with clear instructions",
        "Solution": "Use explicit constraints"
    },
    {
        "Aspect": "Temperature Range",
        "OpenAI": "0-2",
        "Anthropic": "0-1",
        "Solution": "Normalize temperature values"
    },
    {
        "Aspect": "System Prompt Weight",
        "OpenAI": "Moderate influence",
        "Anthropic": "Strong influence",
        "Solution": "Test system prompt effectiveness"
    }
])

print("\nMODEL BEHAVIOR DIFFERENCES:")
print("=" * 80)
print(behavior_differences.to_string(index=False))
print("=" * 80)

### Key Takeaways

✓ **Test empirically** - Don't assume same prompt = same behavior  
✓ **Document differences** - Build knowledge base  
✓ **Explicit constraints** - More specific = more consistent  
✓ **Normalize parameters** - Handle temperature range differences

## CHALLENGE 2: PROMPT ADAPTATION STRATEGIES

**Time:** 10 minutes  
**Objective:** Create prompt adapters for different providers

### Background

Instead of maintaining separate prompts for each provider, create **adapters** that modify prompts based on known model behaviors.

### Student Exercise

In [ ]:
# TODO: Create PromptAdapter class
# Requirements:
# - Takes base prompt
# - Applies provider-specific modifications
# - Returns adapted prompt

class PromptAdapter:
    """
    Adapts prompts for different LLM providers
    """
    
    def __init__(self, base_prompt: str):
        """
        Args:
            base_prompt (str): Base prompt template
        """
        # TODO: Implement
        pass
    
    def adapt_for_provider(self, provider: str) -> str:
        """
        Adapt prompt for specific provider
        
        Args:
            provider (str): 'openai', 'anthropic', etc.
        
        Returns:
            str: Adapted prompt
        """
        # TODO: Implement
        pass

# TODO: Test adapter

### Solution

In [ ]:
# SOLUTION: PromptAdapter with provider-specific rules

class PromptAdapter:
    """
    Adapts prompts for different LLM providers based on known behaviors
    """
    
    # Provider-specific adaptation rules
    ADAPTATION_RULES = {
        "openai": {
            "json_enforcement": "Return ONLY valid JSON with no markdown formatting.",
            "conciseness": "",  # OpenAI is concise by default
            "instruction_prefix": "",
            "temperature_max": 2.0
        },
        "anthropic": {
            "json_enforcement": "You must return ONLY valid JSON. Do not include any preamble, explanation, or markdown. Begin your response with {{ and end with }}.",
            "conciseness": "Be concise and direct in your response.",
            "instruction_prefix": "You must follow these instructions exactly:",
            "temperature_max": 1.0
        },
        "azure_openai": {
            "json_enforcement": "Return ONLY valid JSON with no markdown formatting.",
            "conciseness": "",
            "instruction_prefix": "",
            "temperature_max": 2.0
        }
    }
    
    def __init__(self, base_prompt: str, prompt_type: str = "general"):
        """
        Initialize adapter
        
        Args:
            base_prompt (str): Base prompt (provider-agnostic)
            prompt_type (str): Type of prompt (general, json, classification, etc.)
        """
        self.base_prompt = base_prompt
        self.prompt_type = prompt_type
    
    def adapt_for_provider(self, provider: str, **kwargs) -> str:
        """
        Adapt prompt for specific provider
        
        Args:
            provider (str): Provider name
            **kwargs: Additional parameters (e.g., enforce_json=True)
        
        Returns:
            str: Adapted prompt
        """
        if provider not in self.ADAPTATION_RULES:
            # Default to base prompt for unknown providers
            return self.base_prompt
        
        rules = self.ADAPTATION_RULES[provider]
        adapted = self.base_prompt
        
        # Apply JSON enforcement if needed
        if kwargs.get("enforce_json", False) and self.prompt_type == "json":
            adapted = rules["json_enforcement"] + "\n\n" + adapted
        
        # Apply conciseness instruction if needed
        if kwargs.get("enforce_conciseness", False) and rules["conciseness"]:
            adapted = rules["conciseness"] + "\n\n" + adapted
        
        # Add instruction prefix if present
        if rules["instruction_prefix"]:
            adapted = rules["instruction_prefix"] + "\n\n" + adapted
        
        return adapted
    
    def normalize_temperature(self, temperature: float, provider: str) -> float:
        """
        Normalize temperature for provider's range
        
        Args:
            temperature (float): Desired temperature (0-2 scale)
            provider (str): Provider name
        
        Returns:
            float: Normalized temperature
        """
        if provider not in self.ADAPTATION_RULES:
            return temperature
        
        max_temp = self.ADAPTATION_RULES[provider]["temperature_max"]
        
        # If temperature exceeds provider max, scale it down
        if temperature > max_temp:
            return max_temp
        
        return temperature

# Test the adapter
print("PROMPT ADAPTER TEST:")
print("=" * 80)

# Base prompt (provider-agnostic)
base_fraud_prompt = """
Analyze this transaction for fraud indicators:

Transaction: Card {card}, ${amount} at {merchant}, {location}, {time}

Return a JSON object with:
- risk_level: HIGH, MEDIUM, or LOW
- reasoning: Brief explanation
- recommendation: Action to take
"""

# Create adapter
adapter = PromptAdapter(base_fraud_prompt, prompt_type="json")

# Test data
test_data = {
    "card": "4523",
    "amount": "1250",
    "merchant": "Electronics Store",
    "location": "Lagos, Nigeria",
    "time": "3:47 AM"
}

# Adapt for OpenAI
openai_adapted = adapter.adapt_for_provider("openai", enforce_json=True)
print("\nOPENAI ADAPTED PROMPT:")
print("-" * 80)
print(openai_adapted.format(**test_data))

# Adapt for Anthropic
anthropic_adapted = adapter.adapt_for_provider("anthropic", enforce_json=True)
print("\n" + "=" * 80)
print("\nANTHROPIC ADAPTED PROMPT:")
print("-" * 80)
print(anthropic_adapted.format(**test_data))

# Test temperature normalization
print("\n" + "=" * 80)
print("\nTEMPERATURE NORMALIZATION:")
print("-" * 80)

for temp in [0, 0.7, 1.5]:
    openai_temp = adapter.normalize_temperature(temp, "openai")
    anthropic_temp = adapter.normalize_temperature(temp, "anthropic")
    print(f"Input: {temp} → OpenAI: {openai_temp}, Anthropic: {anthropic_temp}")

print("=" * 80)

### Key Takeaways

✓ **Single base prompt** - Maintain one source of truth  
✓ **Provider-specific adaptations** - Automatic modifications  
✓ **Testable** - Verify adaptations work  
✓ **Maintainable** - Update rules in one place

## CHALLENGE 3: PROVIDER ABSTRACTION LAYER

**Time:** 10 minutes  
**Objective:** Build unified interface for multiple providers

### Background

Your application code shouldn't know which provider it's using. Create an abstraction layer that provides a consistent interface.

### Student Exercise

In [ ]:
# TODO: Create UnifiedLLMClient
# Requirements:
# - Single interface for all providers
# - Automatic prompt adaptation
# - Fallback support
# - Consistent response format

class UnifiedLLMClient:
    """
    Unified client for multiple LLM providers
    """
    
    def __init__(self, primary_provider: str, fallback_providers: List[str] = None):
        """
        Args:
            primary_provider (str): Primary provider to use
            fallback_providers (List[str]): Fallback providers if primary fails
        """
        # TODO: Implement
        pass
    
    def complete(self, prompt: str, **kwargs) -> Dict:
        """
        Get completion from LLM
        
        Args:
            prompt (str): Prompt text
            **kwargs: Additional parameters
        
        Returns:
            Dict: Standardized response
        """
        # TODO: Implement
        pass

# TODO: Test unified client

### Solution

In [ ]:
# SOLUTION: Unified LLM Client with abstraction

class UnifiedLLMClient:
    """
    Provider-agnostic LLM client with fallback support
    """
    
    PROVIDER_CLIENTS = {
        "openai": call_openai,
        "anthropic": call_anthropic
    }
    
    def __init__(
        self,
        primary_provider: str,
        fallback_providers: List[str] = None,
        auto_adapt: bool = True
    ):
        """
        Initialize unified client
        
        Args:
            primary_provider (str): Primary provider
            fallback_providers (List[str]): Fallback providers
            auto_adapt (bool): Automatically adapt prompts for providers
        """
        self.primary_provider = primary_provider
        self.fallback_providers = fallback_providers or []
        self.auto_adapt = auto_adapt
        
        # Validate providers
        all_providers = [primary_provider] + self.fallback_providers
        for provider in all_providers:
            if provider not in self.PROVIDER_CLIENTS:
                raise ValueError(f"Unknown provider: {provider}")
    
    def complete(
        self,
        prompt: str,
        system_prompt: str = "You are a helpful AI assistant.",
        temperature: float = 0,
        prompt_type: str = "general",
        **kwargs
    ) -> Dict:
        """
        Get completion from LLM with fallback support
        
        Args:
            prompt (str): User prompt
            system_prompt (str): System prompt
            temperature (float): Sampling temperature
            prompt_type (str): Type of prompt for adaptation
            **kwargs: Provider-specific parameters
        
        Returns:
            Dict: Standardized response with metadata
        """
        # Create adapter if auto_adapt is enabled
        adapter = PromptAdapter(prompt, prompt_type) if self.auto_adapt else None
        
        # Try primary provider
        result = self._try_provider(
            self.primary_provider,
            prompt,
            system_prompt,
            temperature,
            adapter,
            **kwargs
        )
        
        if result['success']:
            result['provider_used'] = self.primary_provider
            result['fallback_attempted'] = False
            return result
        
        # Try fallback providers
        for fallback_provider in self.fallback_providers:
            result = self._try_provider(
                fallback_provider,
                prompt,
                system_prompt,
                temperature,
                adapter,
                **kwargs
            )
            
            if result['success']:
                result['provider_used'] = fallback_provider
                result['fallback_attempted'] = True
                result['primary_provider_failed'] = self.primary_provider
                return result
        
        # All providers failed
        return {
            "success": False,
            "error": "All providers failed",
            "provider_used": None,
            "content": ""
        }
    
    def _try_provider(
        self,
        provider: str,
        prompt: str,
        system_prompt: str,
        temperature: float,
        adapter: Optional[PromptAdapter],
        **kwargs
    ) -> Dict:
        """
        Try a single provider
        
        Args:
            provider (str): Provider name
            prompt (str): User prompt
            system_prompt (str): System prompt
            temperature (float): Temperature
            adapter (PromptAdapter): Optional adapter
            **kwargs: Additional parameters
        
        Returns:
            Dict: Response
        """
        # Adapt prompt if adapter provided
        if adapter:
            adapted_prompt = adapter.adapt_for_provider(provider, **kwargs)
            adapted_temp = adapter.normalize_temperature(temperature, provider)
        else:
            adapted_prompt = prompt
            adapted_temp = temperature
        
        # Call provider
        provider_function = self.PROVIDER_CLIENTS[provider]
        return provider_function(adapted_prompt, system_prompt, temperature=adapted_temp)

# Test unified client
print("UNIFIED LLM CLIENT TEST:")
print("=" * 80)

# Initialize client with fallback
if anthropic_client:
    client = UnifiedLLMClient(
        primary_provider="openai",
        fallback_providers=["anthropic"],
        auto_adapt=True
    )
else:
    client = UnifiedLLMClient(
        primary_provider="openai",
        auto_adapt=True
    )

# Test prompt
test_prompt = """
Classify this customer inquiry:

"I want to increase my credit card limit"

Categories: CARD_SERVICES, LENDING, FRAUD, ACCOUNT_INQUIRY, GENERAL

Output ONLY the category name.
"""

# Make request (will use primary provider)
print("\nTest 1: Normal request")
response1 = client.complete(
    test_prompt,
    system_prompt="You are a classification system.",
    prompt_type="classification"
)

print(f"Success: {response1['success']}")
print(f"Provider used: {response1.get('provider_used', 'N/A')}")
print(f"Response: {response1['content']}")

print("=" * 80)

### Key Takeaways

✓ **Abstraction** - Hide provider details  
✓ **Consistency** - Same interface for all providers  
✓ **Reliability** - Automatic failover  
✓ **Maintainability** - Single place to update logic

## LAB SUMMARY

### Multi-Provider Patterns Mastered

| Challenge | Pattern | Key Benefit | Production Impact |
|-----------|---------|-------------|-------------------|
| 1 | Behavior Analysis | Document provider differences | Informed design decisions |
| 2 | Prompt Adaptation | Auto-adapt for each provider | Single prompt, multiple providers |
| 3 | Abstraction Layer | Unified interface | Provider independence |

### Multi-Provider Architecture

```
Application Layer
    ↓
Template Library
    ↓
Prompt Adapter (provider-specific rules)
    ↓
Unified LLM Client (abstraction)
    ↓
Provider Clients (OpenAI, Anthropic, Azure, etc.)
    ↓
LLM APIs
```

### Production Checklist

Before deploying multi-provider system:

- [ ] Documented provider behavior differences
- [ ] Created prompt adapters for each provider
- [ ] Built unified client interface
- [ ] Tested prompts across all providers
- [ ] Implemented fallback logic
- [ ] Set up monitoring and metrics
- [ ] Configured cost tracking per provider
- [ ] Established SLAs and error handling
- [ ] Created runbooks for provider failures
- [ ] Documented provider switch procedures

